# Simple Keras

参考：[Keras - straightforward](https://www.kaggle.com/mihaskalic/keras-straightforward)


In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import log_loss

train = pd.read_json("../ShipIceberg/Data/train.json")
#test_df = pd.read_json("../ShipIceberg/Data/test.json")

import json
with open('../ShipIceberg/Data/test.json', 'r') as f:
    test = json.load(f)
    test=pd.DataFrame(test)
    
print('done!')

done!


In [2]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
# test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_test_angle=test['inc_angle']

In [3]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis], x_band2[:, :, :, np.newaxis]], axis=-1)
y_train = np.array(train["is_iceberg"])
print("Xtrain:", X_train.shape)

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis], x_band2[:, :, :, np.newaxis]], axis=-1)
print("Xtest:", X_test.shape)

Xtrain: (1604, 75, 75, 2)
Xtest: (8424, 75, 75, 2)


In [4]:
def get_stats(data,label=1):
    data['max'+str(label)] = [np.max(np.array(x)) for x in data['band_'+str(label)] ]
    data['maxpos'+str(label)] = [np.argmax(np.array(x)) for x in data['band_'+str(label)] ]
    data['min'+str(label)] = [np.min(np.array(x)) for x in data['band_'+str(label)] ]
    data['minpos'+str(label)] = [np.argmin(np.array(x)) for x in data['band_'+str(label)] ]
    data['med'+str(label)] = [np.median(np.array(x)) for x in data['band_'+str(label)] ]
    data['std'+str(label)] = [np.std(np.array(x)) for x in data['band_'+str(label)] ]
    data['mean'+str(label)] = [np.mean(np.array(x)) for x in data['band_'+str(label)] ]
    data['p25_'+str(label)] = [np.sort(np.array(x))[int(0.25*75*75)] for x in data['band_'+str(label)] ]
    data['p75_'+str(label)] = [np.sort(np.array(x))[int(0.75*75*75)] for x in data['band_'+str(label)] ]
    data['mid50_'+str(label)] = data['p75_'+str(label)]-data['p25_'+str(label)]

    return data

train = get_stats(train,1)
train = get_stats(train,2)
X_min1 = train['min1']
X_max1 = train['max1']
X_std1 = train['std1']
X_med1 = train['med1']
X_mean1 = train['mean1']
X_min2 = train['min2']
X_max2 = train['max2']
X_std2 = train['std2']
X_med2 = train['med2']
X_mean2 = train['mean2']

test = get_stats(test,1)
test = get_stats(test,2)
X_test_min1 = test['min1']
X_test_max1 = test['max1']
X_test_std1 = test['std1']
X_test_med1 = test['med1']
X_test_mean1 = test['mean1']
X_test_min2 = test['min2']
X_test_max2 = test['max2']
X_test_std2 = test['std2']
X_test_med2 = test['med2']
X_test_mean2 = test['mean2']

In [5]:
#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D,Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers

from keras.optimizers import Adam
#from keras.optimizers import rmsprop
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad
from keras.optimizers import Adadelta
from keras.optimizers import Adamax
from keras.optimizers import Nadam


from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.layers import Concatenate, Dense, Input, concatenate
from keras.preprocessing import image

# ResNet

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator

# 批处理大小的设置
batch_size=64
# batch_size=16

# Define the image transformations here
# 原始旋转范围10
#gen = ImageDataGenerator(horizontal_flip = True,
#                         vertical_flip = True,
#                         width_shift_range = 0.,
#                         height_shift_range = 0.,
#                         channel_shift_range=0,
#                         zoom_range = 0.2,
#                         rotation_range = 10)



# aug1
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0.,
                         zoom_range = 0.1,
                         rotation_range = 10)

'''
# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]
            '''

def gen_flow_for_twelve_inputs(X1, X2, X3,X4,X5,X6,X7,X8,X9,X10,X11,X12, y):
# def gen_flow_for_twelve_inputs(X1, y):
    gseed = 55 # 原来55
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=gseed)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=gseed)
    genX3 = gen.flow(X1,X3, batch_size=batch_size,seed=gseed)
    genX4 = gen.flow(X1,X4, batch_size=batch_size,seed=gseed)
    genX5 = gen.flow(X1,X5, batch_size=batch_size,seed=gseed)
    genX6 = gen.flow(X1,X6, batch_size=batch_size,seed=gseed)
    genX7 = gen.flow(X1,X7, batch_size=batch_size,seed=gseed)
    genX8 = gen.flow(X1,X8, batch_size=batch_size,seed=gseed)
    genX9 = gen.flow(X1,X9, batch_size=batch_size,seed=gseed)
    genX10 = gen.flow(X1,X10, batch_size=batch_size,seed=gseed)
    genX11 = gen.flow(X1,X11, batch_size=batch_size,seed=gseed)
    genX12 = gen.flow(X1,X12, batch_size=batch_size,seed=gseed)
#     genX13 = gen.flow(X1,X13, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            
            X2i = genX2.next()
            X3i = genX3.next()
            X4i = genX4.next()
            X5i = genX5.next()
            X6i = genX6.next()
            X7i = genX7.next()
            X8i = genX8.next()
            X9i = genX9.next()
            X10i = genX10.next()
            X11i = genX11.next()
            X12i = genX12.next()
            
#             X13i = genX13.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1],X3i[1],X4i[1],X5i[1],X6i[1],X7i[1],
                  X8i[1],X9i[1],X10i[1],X11i[1],X12i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


Using TensorFlow backend.


In [6]:
def getVggAngleModel():
    
    """
    Keras Sequential model

    """
#     input_2 = Input(shape=[1], name="angle")
#     angle_layer = Dense(1, )(input_2)
    input_1 = Input(shape=(75,75,2))
    
    input_2 = Input(shape=[1], name="angle")
    input_3 = Input(shape=[1], name="min1")
    input_4 = Input(shape=[1], name="max1")
    input_5 = Input(shape=[1], name="std1")
    input_6 = Input(shape=[1], name="med1")
    input_7 = Input(shape=[1], name="mean1")
    input_8 = Input(shape=[1], name="min2")
    input_9 = Input(shape=[1], name="max2")
    input_10 = Input(shape=[1], name="std2")
    input_11 = Input(shape=[1], name="med2")
    input_12 = Input(shape=[1], name="mean2")
#     input_13 = Input(shape=[1], name="size")
    
    angle_layer = Dense(1, )(input_2)
    min1_layer = Dense(1, )(input_3)
    max1_layer = Dense(1, )(input_4)
    std1_layer = Dense(1, )(input_5)
    med1_layer = Dense(1, )(input_6)
    mean1_layer = Dense(1, )(input_7)
    min2_layer = Dense(1, )(input_8)
    max2_layer = Dense(1, )(input_9)
    std2_layer = Dense(1, )(input_10)
    med2_layer = Dense(1, )(input_11)
    mean2_layer = Dense(1, )(input_12)
#     size_layer = Dense(1, )(input_13)
    
#     model=Sequential()
    
    # Conv block 1
    x=Conv2D(64, kernel_size=(3, 3),activation='relu')(input_1)
    x=Conv2D(64, kernel_size=(3, 3), activation='relu' )(x)
    x=Conv2D(64, kernel_size=(3, 3), activation='relu' )(x)
    x=MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
   
    # Conv block 2
    x=Conv2D(128, kernel_size=(3, 3), activation='relu' )(x)
    x=Conv2D(128, kernel_size=(3, 3), activation='relu' )(x)
    x=Conv2D(128, kernel_size=(3, 3), activation='relu' )(x)
    x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
   
    # Conv block 3
    x=Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
   
    #Conv block 4
    x=Conv2D(256, kernel_size=(3, 3), activation='relu')(x)
    x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
   
    # Flatten before dense
    x=Flatten()(x)
    
#     merge_one = concatenate([x, angle_layer])
    merge_one = concatenate([x, angle_layer,
                             min1_layer,max1_layer,std1_layer,med1_layer,mean1_layer,
                            min2_layer,max2_layer,std2_layer,med2_layer,mean2_layer])

    #Dense 1
    merge_one = Dense(1024, activation='relu')(merge_one)
    merge_one = Dropout(0.4)(merge_one)

    #Dense 2
    merge_one = Dense(512, activation='relu')(merge_one)
    merge_one = Dropout(0.2)(merge_one)

    # Output 
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
#     model = Model(input=[input_1, input_2], output=predictions)
    model = Model(input=[input_1, input_2,input_3,input_4,input_5,input_6,
                        input_7,input_8,input_9,input_10,input_11,input_12], 
                  output=predictions)

    optimizer = Adam(lr=0.0001, decay=0.0)
    sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    return model

In [7]:
# Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, 
              X_min1,X_max1,X_std1,X_med1,X_mean1,
              X_min2,X_max2,X_std2,X_med2,X_mean2,
              X_test):
    # K-折交叉验证
    K=4
    
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=2017).split(X_train, target_train)) # 原16
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]
        
        X_min1_cv=X_min1[train_idx]
        X_min1_hold=X_min1[test_idx]
        
        X_max1_cv=X_max1[train_idx]
        X_max1_hold=X_max1[test_idx]
        
        X_std1_cv=X_std1[train_idx]
        X_std1_hold=X_std1[test_idx]
        
        X_med1_cv=X_med1[train_idx]
        X_med1_hold=X_med1[test_idx]
        
        X_mean1_cv=X_mean1[train_idx]
        X_mean1_hold=X_mean1[test_idx]
        
        X_min2_cv=X_min2[train_idx]
        X_min2_hold=X_min2[test_idx]
        
        X_max2_cv=X_max2[train_idx]
        X_max2_hold=X_max2[test_idx]
        
        X_std2_cv=X_std2[train_idx]
        X_std2_hold=X_std2[test_idx]
        
        X_med2_cv=X_med2[train_idx]
        X_med2_hold=X_med2[test_idx]
        
        X_mean2_cv=X_mean2[train_idx]
        X_mean2_hold=X_mean2[test_idx]
        
        
        
        
        

        #define file path and get callbacks
        file_path = "./model/%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_twelve_inputs(X_train_cv, X_angle_cv, 
                                              X_min1_cv,X_max1_cv,X_std1_cv,X_med1_cv,X_mean1_cv,
                                              X_min2_cv,X_max2_cv,X_std2_cv,X_med2_cv,X_mean2_cv,
                                              y_train_cv)
#         gen_flow = gen_flow_for_twelve_inputs(X_train_cv,y_train_cv)
    
        galaxyModel= getVggAngleModel()
        
#         gen_flow1=[gen_flow,X_angle_cv, X_min1_cv,X_max1_cv,X_std1_cv,X_med1_cv,X_mean1_cv,
#                    X_min2_cv,X_max2_cv,X_std2_cv,X_med2_cv,X_mean2_cv]
        
        # 调整训练参数
        galaxyModel.fit_generator(
                gen_flow,
#                 steps_per_epoch=24,
                steps_per_epoch = len(X_train_cv)//batch_size,
                #steps_per_epoch=100,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold,
                                 X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
                                 X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold], 
                                 Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv,
                                     X_min1_cv,X_max1_cv,X_std1_cv,X_med1_cv,X_mean1_cv,
                                     X_min2_cv,X_max2_cv,X_std2_cv,X_med2_cv,X_mean2_cv], 
                                     y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold,
                                     X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
                                     X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold], 
                                     Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold,
                                       X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
                                       X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle,
                                      X_test_min1,X_test_max1,X_test_std1,X_test_med1,X_test_mean1,
                                      X_test_min2,X_test_max2,X_test_std2,X_test_med2,X_test_mean2])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle,
                                       X_min1,X_max1,X_std1,X_med1,X_mean1,
                                       X_min2,X_max2,X_std2,X_med2,X_mean2])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_valid_pred_log,y_test_pred_log

In [8]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import log_loss

# preds=myAngleCV(X_train, X_angle, X_test)
train_preds,test_preds=myAngleCV(X_train, X_angle, 
                X_min1,X_max1,X_std1,X_med1,X_mean1,
                X_min2,X_max2,X_std2,X_med2,X_mean2,
                X_test)


===================FOLD= 0


/home/jayden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:81: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100


/home/jayden/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:787: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (1202, 75, 75, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


18/18 [==============================] - 9s - loss: 0.8994 - acc: 0.5104 - val_loss: 0.6392 - val_acc: 0.5323
Epoch 2/100
18/18 [==============================] - 6s - loss: 0.8325 - acc: 0.5383 - val_loss: 0.6023 - val_acc: 0.6418
Epoch 3/100
18/18 [==============================] - 6s - loss: 0.7453 - acc: 0.6005 - val_loss: 0.5996 - val_acc: 0.6468
Epoch 4/100
18/18 [==============================] - 6s - loss: 0.7237 - acc: 0.6006 - val_loss: 0.5879 - val_acc: 0.6318
Epoch 5/100
18/18 [==============================] - 6s - loss: 0.7177 - acc: 0.6012 - val_loss: 0.5875 - val_acc: 0.6393
Epoch 6/100
18/18 [==============================] - 6s - loss: 0.7257 - acc: 0.5967 - val_loss: 0.5874 - val_acc: 0.6468
Epoch 7/100
18/18 [==============================] - 6s - loss: 0.6913 - acc: 0.6408 - val_loss: 0.5840 - val_acc: 0.6468
Epoch 8/100
18/18 [==============================] - 6s - loss: 0.7067 - acc: 0.6004 - val_loss: 0.5798 - val_acc: 0.6517
Epoch 9/100
18/18 [=================

18/18 [==============================] - 8s - loss: 0.5788 - acc: 0.6679 - val_loss: 0.5433 - val_acc: 0.7065
Epoch 68/100
18/18 [==============================] - 8s - loss: 0.5555 - acc: 0.6965 - val_loss: 0.5348 - val_acc: 0.7114
Epoch 69/100
18/18 [==============================] - 6s - loss: 0.5643 - acc: 0.6860 - val_loss: 0.5406 - val_acc: 0.7090
Epoch 70/100
18/18 [==============================] - 7s - loss: 0.5674 - acc: 0.6742 - val_loss: 0.5280 - val_acc: 0.7040
Epoch 71/100
18/18 [==============================] - 7s - loss: 0.5766 - acc: 0.6677 - val_loss: 0.5407 - val_acc: 0.7090
Epoch 72/100
18/18 [==============================] - 7s - loss: 0.5528 - acc: 0.6980 - val_loss: 0.5309 - val_acc: 0.7090
Epoch 73/100
18/18 [==============================] - 8s - loss: 0.5659 - acc: 0.6916 - val_loss: 0.5370 - val_acc: 0.7090
Epoch 74/100
18/18 [==============================] - 7s - loss: 0.5770 - acc: 0.6651 - val_loss: 0.5320 - val_acc: 0.7139
Epoch 75/100
18/18 [=========

/home/jayden/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:787: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (1203, 75, 75, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


18/18 [==============================] - 8s - loss: 1.0235 - acc: 0.5469 - val_loss: 0.6291 - val_acc: 0.6883
Epoch 2/100
18/18 [==============================] - 6s - loss: 0.9033 - acc: 0.5601 - val_loss: 0.5875 - val_acc: 0.6683
Epoch 3/100
18/18 [==============================] - 7s - loss: 0.8603 - acc: 0.5891 - val_loss: 0.5895 - val_acc: 0.6733
Epoch 4/100
18/18 [==============================] - 7s - loss: 0.8321 - acc: 0.5928 - val_loss: 0.5813 - val_acc: 0.6608
Epoch 5/100
18/18 [==============================] - 7s - loss: 0.8175 - acc: 0.5789 - val_loss: 0.5800 - val_acc: 0.6459
Epoch 6/100
18/18 [==============================] - 7s - loss: 0.7546 - acc: 0.6210 - val_loss: 0.5739 - val_acc: 0.6633
Epoch 7/100
18/18 [==============================] - 7s - loss: 0.7311 - acc: 0.6169 - val_loss: 0.5737 - val_acc: 0.6683
Epoch 8/100
18/18 [==============================] - 7s - loss: 0.7217 - acc: 0.6241 - val_loss: 0.5803 - val_acc: 0.6708
Epoch 9/100
18/18 [=================

/home/jayden/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:787: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (1204, 75, 75, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/100
18/18 [==============================] - 8s - loss: 1.2320 - acc: 0.4696 - val_loss: 0.6923 - val_acc: 0.5125
Epoch 2/100
18/18 [==============================] - 6s - loss: 0.9808 - acc: 0.5293 - val_loss: 0.6012 - val_acc: 0.6725
Epoch 3/100
18/18 [==============================] - 7s - loss: 0.8802 - acc: 0.5784 - val_loss: 0.5908 - val_acc: 0.6550
Epoch 4/100
18/18 [==============================] - 8s - loss: 0.8098 - acc: 0.5858 - val_loss: 0.5901 - val_acc: 0.6700
Epoch 5/100
18/18 [==============================] - 7s - loss: 0.7480 - acc: 0.6293 - val_loss: 0.5828 - val_acc: 0.6600
Epoch 6/100
18/18 [==============================] - 6s - loss: 0.7690 - acc: 0.6128 - val_loss: 0.5797 - val_acc: 0.6600
Epoch 7/100
18/18 [==============================] - 7s - loss: 0.6926 - acc: 0.6497 - val_loss: 0.5826 - val_acc: 0.6650
Epoch 8/100
18/18 [==============================] - 7s - loss: 0.6728 - acc: 0.6526 - val_loss: 0.5893 - val_acc: 0.6750
Epoch 9/100
18/18 [=====